In [ ]:
dimension = 512

build_index_parameters = dict()
distance_metric = 'l2'

index_parameters = {'R': '64', 'L': '100', 'M': '16', 'B': '0.45'}

B_value  = (0.25 + (250000*(4*int(index_parameters['R']) + 4*dimension)) / 2 ** 30)
index_parameters['B'] = str(B_value)[:4]

print("B value: ", index_parameters['B'])


B value:  0.78


# download vector files from the h100-dell

In [5]:
import subprocess
target_train_fvecs = '<target_location>:/local/eric/ft-vect-train/ft_size_' + str(dimension) + '/' + 'ft_' + str(dimension) + '.pack'
target_validation_fvecs = '<target_location>:/local/eric/ft-vect-validation/ft_size_' + str(dimension) + '/' + 'ft_' + str(dimension) + '.pack'
train_fvecs_name = 'ft_train_' + str(dimension) + '.pack'
validation_fvecs_name = 'ft_validation_' + str(dimension) + '.pack'
result = subprocess.run(["rm", "-r", "data/tmp/"], capture_output=True, text=True)
result = subprocess.run(["mkdir", "data/tmp/"], capture_output=True, text=True)

result = subprocess.run(['scp', target_train_fvecs, 'data/tmp/'+train_fvecs_name], capture_output=True, text=True)
result = subprocess.run(['scp', target_validation_fvecs, 'data/tmp/'+validation_fvecs_name], capture_output=True, text=True)
result = subprocess.run(['ls', 'data/tmp/'], capture_output=True, text=True)

In [ ]:
# convert the fvecs to fbin
target_index_fbin = "ft_index_" + str(dimension) + ".fbin"
target_query_fbins = "ft_query_" + str(dimension) + ".fbin"
result = subprocess.run(["./apps/utils/fvecs_to_bin", "float", "data/tmp/"+train_fvecs_name, "data/tmp/"+target_index_fbin], capture_output=True, text=True)
print(result.stdout)
result = subprocess.run(["./apps/utils/fvecs_to_bin", "float", "data/tmp/"+validation_fvecs_name, "data/tmp/"+target_query_fbins], capture_output=True, text=True)
print(result.stdout)
#result = subprocess.run(['rm', 'data/tmp/'+train_fvecs_name], capture_output=True, text=True)
#result = subprocess.run(['rm', 'data/tmp/'+validation_fvecs_name], capture_output=True, text=True)

# compute the ground truth

Dataset: #pts = 1281167, # dims = 512
# blks: 10
Block #0 written
Block #1 written
Block #2 written
Block #3 written
Block #4 written
Block #5 written
Block #6 written
Block #7 written
Block #8 written
Block #9 written

Dataset: #pts = 50000, # dims = 512
# blks: 1
Block #0 written




In [9]:
# compute the ground truth
gt_file_name = "ft_gt_" + str(dimension) + "_gt1000"
result = subprocess.run(["./apps/utils/compute_groundtruth", "--data_type", "float", "--dist_fn", distance_metric, "--base_file", "data/tmp/"+target_index_fbin, "--query_file", "data/tmp/"+target_query_fbins, "--gt_file", "data/tmp/"+gt_file_name, "--K", "10"], capture_output=True, text=True)
print(result)

CompletedProcess(args=['./apps/utils/compute_groundtruth', '--data_type', 'float', '--dist_fn', 'ls', '--base_file', 'data/tmp/ft_index_512.fbin', '--query_file', 'data/tmp/ft_query_512.fbin', '--gt_file', 'data/tmp/ft_gt_512_gt1000', '--K', '10'], returncode=255, stdout='', stderr='Unsupported distance function. Use l2/mips/cosine.\n')


In [9]:
graph_index = "ft_graph_" + str(dimension)
result = subprocess.run(['./apps/build_disk_index', '--data_type', 'float', '--dist_fn', distance_metric, '--data_path', 'data/tmp/' + target_index_fbin, '--index_path_prefix', 'data/tmp/' + graph_index, '-R', index_parameters['R'], '-L'+index_parameters['L'], '-B' , index_parameters['B'], '-M', index_parameters['M']], capture_output=True, text=True)
print(result.stdout)

Normalizing data for cosine to temporary file, please ensure there is additional (n*d*4) bytes for storing normalized base vectors, apart from the interim indices created by DiskANN and the final index.
Normalizing FLOAT vectors in file: data/tmp/ft_index_8.fbin
Dataset: #pts = 1281167, # dims = 8
# blks: 10
Wrote normalized points to file: data/tmp/ft_graph_8_prepped_base.bin
Time for preprocessing data for cosine: 0.038088 seconds
Starting index build: R=64 L=100 Query RAM budget: 3.3286e+08 Indexing ram budget: 8 T: 12
Compressing 8-dimensional data into 8 bytes per vector.
Opened: data/tmp/ft_graph_8_prepped_base.bin, size: 40997352, cache_size: 40997352
Training data with 256393 samples loaded.
Processing chunk 0 with dimensions [0, 1)
Processing chunk 1 with dimensions [1, 2)
Processing chunk 2 with dimensions [2, 3)
Processing chunk 3 with dimensions [3, 4)
Processing chunk 4 with dimensions [4, 5)
Processing chunk 5 with dimensions [5, 6)
Processing chunk 6 with dimensions [6, 

In [12]:
result_path = "data/tmp/res"
result = subprocess.run(['./apps/search_disk_index', '--data_type', 'float', '--dist_fn', distance_metric, '--index_path_prefix', 'data/tmp/' + graph_index, '--query_file', 'data/tmp/' + target_query_fbins, '--gt_file', 'data/tmp/' + gt_file_name, '--result_path', result_path, '-K', '10', '-L', '10', '20', '30', '40', '50', '100', '--num_nodes_to_cache', '10000'], capture_output=True, text=True)
print(result.stdout)
#write stdout to file
with open('data/tmp/result.txt', 'w') as f:
    f.write(result.stdout)


Search parameters: #threads: 12,  beamwidth: 2.
Reading (with alignment) bin file data/tmp/ft_query_8.fbin ...[FUNC: void diskann::load_aligned_bin(const string&, T*&, size_t&, size_t&, size_t&) [with T = float; std::string = std::__cxx11::basic_string<char>; size_t = long unsigned int]][FILE: /home/xiangyu/working_dir/Utoronto/DiskANN/include/utils.h][LINE: 825]   While opening file 'data/tmp/ft_query_8.fbin', error code: 1  iostream error



FileNotFoundError: [Errno 2] No such file or directory: 'data/tmp/result.txt'

In [11]:
subprocess.run(['mv', 'data/tmp/', 'data/MRL_' + str(dimension) + '_R' + index_parameters['R'] + '_L' + index_parameters['L'] + '_M' + index_parameters['M'] + '_B' + index_parameters['B'] + "_" + distance_metric])

mv: cannot stat 'data/tmp/'

CompletedProcess(args=['mv', 'data/tmp/', 'data/MRL_8_R64_L100_M8_B0.31_cosine'], returncode=1)

: No such file or directory
